In [1]:
!pip install openpyxl
import pandas as pd

In [2]:
#Read ANNEX table from UN_MigrantStockTotal_2015 and convert it into form that is easier to manipulate
ANNEX = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', sheet_name="ANNEX")
ANNEX = ANNEX.iloc[15:]
ANNEX.columns = ["Country code", "Country or area", "Sort order of Country", "Major area", "Code of Major area", 
                "Sort order of Major area", "Region", "Code of Region", "Sort order of Region", "Developed region",
                "Least developed country", "Sub-Saharan Africa"]
ANNEX

Country code            Country or area Sort order of Country Major area  \
15             8                    Albania                   154     Europe   
16            12                    Algeria                    40     Africa   
17            16             American Samoa                   257    Oceania   
18            20                    Andorra                   155     Europe   
19            24                     Angola                    30     Africa   
..           ...                        ...                   ...        ...   
241          876  Wallis and Futuna Islands                   265    Oceania   
242          732             Western Sahara                    46     Africa   
243          887                      Yemen                   126       Asia   
244          894                     Zambia                    27     Africa   
245          716                   Zimbabwe                    28     Africa   

    Code of Major area Sort order of Major area           Region  \
15                 908                      127  Southern Europe   
16                 903                        7  Northern Africa   
17                 909                      238        Polynesia   
18                 908                      127  Southern Europe   
19                 903                        7    Middle Africa   
..                 ...                      ...              ...   
241                909                      238        Polynesia   
242                903                        7  Northern Africa   
243                935                       71     Western Asia   
244                903                        7   Eastern Africa   
245                903                        7   Eastern Africa   

    Code of Region Sort order of Region Developed region  \
15             925                  153              Yes   
16             912                   39               No   
17             957                  256               No   
18             925                  153              Yes   
19             911                   29               No   
..             ...                  ...              ...   
241            957                  256               No   
242            912                   39               No   
243            922                  108               No   
244            910                    8               No   
245            910                    8               No   

    Least developed country Sub-Saharan Africa  
15                       No                 No  
16                       No                 No  
17                       No                 No  
18                       No                 No  
19                      Yes                Yes  
..                      ...                ...  
241                      No                 No  
242                      No                 No  
243                     Yes                 No  
244                     Yes                Yes  
245                      No                Yes  

[231 rows x 12 columns]

In [3]:
#Read table 1 from UN_MigrantStockTotal_2015, remove the first 15 rows that are not data
UN_1 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', sheet_name="Table 1")
UN_1 = UN_1.iloc[15:]

#Rename columns
UN_1.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                'Both-1990','Both-1995','Both-2000','Both-2005','Both-2010','Both-2015',
                'Male-1990','Male-1995','Male-2000','Male-2005','Male-2010','Male-2015',
                'Feml-1990','Feml-1995','Feml-2000','Feml-2005','Feml-2010','Feml-2015']

#First,colums of form (gender-year) are variable values not variable names, thus them violate principle 1.
#Second, those colums have multiple variables: gender and year, which violate principle 2.
#Melt those 18 columns into 3 columns: "gender", "year", and "International migrant stock at mid-year"(from the original table)
UN_1 = UN_1.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code','Type of data (a)'], 
                 var_name=["tmp"],value_name="International migrant stock at mid-year")
UN_1 = UN_1.assign(gender = lambda x: x.tmp.str[0:4].astype(str), 
                    year = lambda x: x.tmp.str[5:].astype(str)).drop("tmp",axis=1)

#Merge UN_1 with ANNEX on "Country code" so that each row have area information and region information
#Using merge keeps rows that are not countries while using join will delete those rows.
UN_1 = UN_1.merge(ANNEX, how='left', on=['Country code'])
UN_1drop = UN_1.drop(columns=['Country or area', 'Sort order of Country', 
       'Code of Major area', 'Sort order of Major area',
       'Code of Region', 'Sort order of Region'])
UN_1 = UN_1drop[UN_1drop["International migrant stock at mid-year"] != ".."]
UN_1.head(15:20)

SyntaxError: invalid syntax (4109222373.py, line 28)

In [9]:
#Reoder the columns
UN_1 = UN_1[['Sort order', 'Major area, region, country or area of destination',
             'Major area', 'Region','Notes', 'Country code', 'Type of data (a)', 'gender', 'year',
             'International migrant stock at mid-year', 'Developed region', 'Least developed country',
             'Sub-Saharan Africa']]
UN_1

Sort order Major area, region, country or area of destination Major area  \
0             1                                              WORLD        NaN   
1             2                                  Developed regions        NaN   
2             3                                 Developing regions        NaN   
3             4                          Least developed countries        NaN   
4             5  Less developed regions excluding least develop...        NaN   
...         ...                                                ...        ...   
4765        261                                              Samoa    Oceania   
4766        262                                            Tokelau    Oceania   
4767        263                                              Tonga    Oceania   
4768        264                                             Tuvalu    Oceania   
4769        265                          Wallis and Futuna Islands    Oceania   

         Region Notes Country code Type of data (a) gender  year  \
0           NaN   NaN          900              NaN   Both  1990   
1           NaN   (b)          901              NaN   Both  1990   
2           NaN   (c)          902              NaN   Both  1990   
3           NaN   (d)          941              NaN   Both  1990   
4           NaN   NaN          934              NaN   Both  1990   
...         ...   ...          ...              ...    ...   ...   
4765  Polynesia   NaN          882                B   Feml  2015   
4766  Polynesia   NaN          772                B   Feml  2015   
4767  Polynesia   NaN          776                B   Feml  2015   
4768  Polynesia   NaN          798                C   Feml  2015   
4769  Polynesia   NaN          876                B   Feml  2015   

     International migrant stock at mid-year Developed region  \
0                                  152563212              NaN   
1                                   82378628              NaN   
2                                   70184584              NaN   
3                                   11075966              NaN   
4                                   59105261              NaN   
...                                      ...              ...   
4765                                  2460.0               No   
4766                                   254.0               No   
4767                                  2604.0               No   
4768                                    63.0               No   
4769                                  1411.0               No   

     Least developed country Sub-Saharan Africa  
0                        NaN                NaN  
1                        NaN                NaN  
2                        NaN                NaN  
3                        NaN                NaN  
4                        NaN                NaN  
...                      ...                ...  
4765                      No                 No  
4766                      No                 No  
4767                      No                 No  
4768                     Yes                 No  
4769                      No                 No  

[4725 rows x 13 columns]

In [5]:
#Similar to UN_1
UN_2 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', sheet_name="Table 2")
UN_2 = UN_2.iloc[15:]
UN_2.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code',
                'Both-1990','Both-1995','Both-2000','Both-2005','Both-2010','Both-2015',
                'Male-1990','Male-1995','Male-2000','Male-2005','Male-2010','Male-2015',
                'Feml-1990','Feml-1995','Feml-2000','Feml-2005','Feml-2010','Feml-2015']
UN_2 = UN_2.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code'], 
                 var_name=["tmp"],value_name="Total population")
UN_2 = UN_2.assign(gender = lambda x: x.tmp.str[0:4].astype(str), 
                    year = lambda x: x.tmp.str[5:].astype(str)).drop("tmp",axis=1)
UN_2 = UN_2.merge(ANNEX, how='left', on=['Country code'])
UN_2drop = UN_2.drop(columns=['Country or area', 'Sort order of Country', 
       'Code of Major area', 'Sort order of Major area',
       'Code of Region', 'Sort order of Region'])
UN_2 = UN_2drop[UN_2drop["Total population"] != ".."]
UN_2 = UN_2[['Sort order', 'Major area, region, country or area of destination',
             'Major area', 'Region','Notes', 'Country code', 'gender', 'year',
             'Total population', 'Developed region', 'Least developed country',
             'Sub-Saharan Africa']]
UN_2

Sort order Major area, region, country or area of destination Major area  \
0             1                                              WORLD        NaN   
1             2                                  Developed regions        NaN   
2             3                                 Developing regions        NaN   
3             4                          Least developed countries        NaN   
4             5  Less developed regions excluding least develop...        NaN   
...         ...                                                ...        ...   
4756        252                   Micronesia (Federated States of)    Oceania   
4760        256                                          Polynesia        NaN   
4763        259                                   French Polynesia    Oceania   
4765        261                                              Samoa    Oceania   
4767        263                                              Tonga    Oceania   

          Region Notes Country code gender  year Total population  \
0            NaN   NaN          900   Both  1990      5309667.699   
1            NaN   (b)          901   Both  1990      1144463.062   
2            NaN   (c)          902   Both  1990      4165204.637   
3            NaN   (d)          941   Both  1990       510057.629   
4            NaN   NaN          934   Both  1990      3655147.008   
...          ...   ...          ...    ...   ...              ...   
4756  Micronesia   NaN          583   Feml  2015            50.95   
4760         NaN   NaN          957   Feml  2015          336.115   
4763   Polynesia   NaN          258   Feml  2015          138.468   
4765   Polynesia   NaN          882   Feml  2015           93.584   
4767   Polynesia   NaN          776   Feml  2015           52.931   

     Developed region Least developed country Sub-Saharan Africa  
0                 NaN                     NaN                NaN  
1                 NaN                     NaN                NaN  
2                 NaN                     NaN                NaN  
3                 NaN                     NaN                NaN  
4                 NaN                     NaN                NaN  
...               ...                     ...                ...  
4756               No                      No                 No  
4760              NaN                     NaN                NaN  
4763               No                      No                 No  
4765               No                      No                 No  
4767               No                      No                 No  

[4386 rows x 12 columns]

In [6]:
#Similar to UN_1
UN_3 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', sheet_name="Table 3")
UN_3 = UN_3.iloc[15:]
UN_3.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                'Both-1990','Both-1995','Both-2000','Both-2005','Both-2010','Both-2015',
                'Male-1990','Male-1995','Male-2000','Male-2005','Male-2010','Male-2015',
                'Feml-1990','Feml-1995','Feml-2000','Feml-2005','Feml-2010','Feml-2015']
UN_3 = UN_3.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code','Type of data (a)'], 
                 var_name=["tmp"],value_name="International migrant stock as a percentage of the total population")
UN_3 = UN_3.assign(gender = lambda x: x.tmp.str[0:4].astype(str), 
                    year = lambda x: x.tmp.str[5:].astype(str)).drop("tmp",axis=1)
UN_3 = UN_3.merge(ANNEX, how='left', on=['Country code'])
UN_3drop = UN_3.drop(columns=['Country or area', 'Sort order of Country', 
       'Code of Major area', 'Sort order of Major area',
       'Code of Region', 'Sort order of Region'])
UN_3 = UN_3drop[UN_3drop["International migrant stock as a percentage of the total population"] != ".."]
UN_3 = UN_3[['Sort order', 'Major area, region, country or area of destination',
             'Major area', 'Region','Notes', 'Country code', 'Type of data (a)', 'gender', 'year',
             'International migrant stock as a percentage of the total population', 'Developed region', 'Least developed country',
             'Sub-Saharan Africa']]
UN_3

Sort order Major area, region, country or area of destination Major area  \
0             1                                              WORLD        NaN   
1             2                                  Developed regions        NaN   
2             3                                 Developing regions        NaN   
3             4                          Least developed countries        NaN   
4             5  Less developed regions excluding least develop...        NaN   
...         ...                                                ...        ...   
4756        252                   Micronesia (Federated States of)    Oceania   
4760        256                                          Polynesia        NaN   
4763        259                                   French Polynesia    Oceania   
4765        261                                              Samoa    Oceania   
4767        263                                              Tonga    Oceania   

          Region Notes Country code Type of data (a) gender  year  \
0            NaN   NaN          900              NaN   Both  1990   
1            NaN   (b)          901              NaN   Both  1990   
2            NaN   (c)          902              NaN   Both  1990   
3            NaN   (d)          941              NaN   Both  1990   
4            NaN   NaN          934              NaN   Both  1990   
...          ...   ...          ...              ...    ...   ...   
4756  Micronesia   NaN          583                B   Feml  2015   
4760         NaN   NaN          957              NaN   Feml  2015   
4763   Polynesia   NaN          258                B   Feml  2015   
4765   Polynesia   NaN          882                B   Feml  2015   
4767   Polynesia   NaN          776                B   Feml  2015   

     International migrant stock as a percentage of the total population  \
0                                               2.87331                    
1                                              7.198015                    
2                                              1.685021                    
3                                              2.171513                    
4                                              1.617042                    
...                                                 ...                    
4756                                           2.518155                    
4760                                           9.925472                    
4763                                            9.33212                    
4765                                           2.628654                    
4767                                           4.919612                    

     Developed region Least developed country Sub-Saharan Africa  
0                 NaN                     NaN                NaN  
1                 NaN                     NaN                NaN  
2                 NaN                     NaN                NaN  
3                 NaN                     NaN                NaN  
4                 NaN                     NaN                NaN  
...               ...                     ...                ...  
4756               No                      No                 No  
4760              NaN                     NaN                NaN  
4763               No                      No                 No  
4765               No                      No                 No  
4767               No                      No                 No  

[4343 rows x 13 columns]

In [7]:
#Similar to UN_1
UN_4 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', sheet_name="Table 4")
UN_4 = UN_4.iloc[15:]
UN_4.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                '1990','1995','2000','2005','2010','2015']

#Unlike UN_1, those columns in UN_4 do not have gender information, so they do not violate principle 2.
UN_4 = UN_4.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code','Type of data (a)'], 
                 var_name=["tmp"],value_name="Female migrants as a percentage of the international migrant stock")
UN_4 = UN_4.assign(year = lambda x: x.tmp.str[0:].astype(str)).drop("tmp",axis=1)
UN_4 = UN_4.merge(ANNEX, how='left', on=['Country code'])
UN_4drop = UN_4.drop(columns=['Country or area', 'Sort order of Country', 
       'Code of Major area', 'Sort order of Major area',
       'Code of Region', 'Sort order of Region'])
UN_4 = UN_4drop[UN_4drop["Female migrants as a percentage of the international migrant stock"] != ".."]
UN_4 = UN_4[['Sort order', 'Major area, region, country or area of destination',
              'Major area', 'Region','Notes', 'Country code', 'Type of data (a)', 'year',
              'Female migrants as a percentage of the international migrant stock', 'Developed region', 
              'Least developed country','Sub-Saharan Africa']]
UN_4

Sort order Major area, region, country or area of destination Major area  \
0             1                                              WORLD        NaN   
1             2                                  Developed regions        NaN   
2             3                                 Developing regions        NaN   
3             4                          Least developed countries        NaN   
4             5  Less developed regions excluding least develop...        NaN   
...         ...                                                ...        ...   
1585        261                                              Samoa    Oceania   
1586        262                                            Tokelau    Oceania   
1587        263                                              Tonga    Oceania   
1588        264                                             Tuvalu    Oceania   
1589        265                          Wallis and Futuna Islands    Oceania   

         Region Notes Country code Type of data (a)  year  \
0           NaN   NaN          900              NaN  1990   
1           NaN   (b)          901              NaN  1990   
2           NaN   (c)          902              NaN  1990   
3           NaN   (d)          941              NaN  1990   
4           NaN   NaN          934              NaN  1990   
...         ...   ...          ...              ...   ...   
1585  Polynesia   NaN          882                B  2015   
1586  Polynesia   NaN          772                B  2015   
1587  Polynesia   NaN          776                B  2015   
1588  Polynesia   NaN          798                C  2015   
1589  Polynesia   NaN          876                B  2015   

     Female migrants as a percentage of the international migrant stock  \
0                                              49.03915                   
1                                             51.123977                   
2                                             46.592099                   
3                                             47.261155                   
4                                             46.466684                   
...                                                 ...                   
1585                                          49.908704                   
1586                                          52.156057                   
1587                                          45.437096                   
1588                                          44.680851                   
1589                                           49.52615                   

     Developed region Least developed country Sub-Saharan Africa  
0                 NaN                     NaN                NaN  
1                 NaN                     NaN                NaN  
2                 NaN                     NaN                NaN  
3                 NaN                     NaN                NaN  
4                 NaN                     NaN                NaN  
...               ...                     ...                ...  
1585               No                      No                 No  
1586               No                      No                 No  
1587               No                      No                 No  
1588               No                     Yes                 No  
1589               No                      No                 No  

[1575 rows x 12 columns]

In [19]:
#Similiar to UN_1
UN_5 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', sheet_name="Table 5")
UN_5 = UN_5.iloc[15:]
UN_5.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                'Both-1990-1995','Both-1995-2000','Both-2000-2005','Both-2005-2010','Both-2010-2015',
                'Male-1990-1995','Male-1995-2000','Male-2000-2005','Male-2005-2010','Male-2010-2015',
                'Feml-1990-1995','Feml-1995-2000','Feml-2000-2005','Feml-2005-2010','Feml-2010-2015',]
UN_5 = UN_5.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code','Type of data (a)'], 
                 var_name=["tmp"],value_name="Annual rate of change of the migrant stock")
UN_5 = UN_5.assign(gender = lambda x: x.tmp.str[0:4].astype(str), 
                    year = lambda x: x.tmp.str[5:].astype(str)).drop("tmp",axis=1)
UN_5 = UN_5.merge(ANNEX, how='left', on=['Country code'])
UN_5drop = UN_5.drop(columns=['Country or area', 'Sort order of Country', 
       'Code of Major area', 'Sort order of Major area',
       'Code of Region', 'Sort order of Region'])
UN_5 = UN_5drop[UN_5drop["Annual rate of change of the migrant stock"] != ".."]
UN_5 = UN_5[['Sort order', 'Major area, region, country or area of destination',
             'Major area', 'Region','Notes', 'Country code', 'Type of data (a)', 'gender', 'year',
             'Annual rate of change of the migrant stock', 'Developed region', 'Least developed country',
             'Sub-Saharan Africa']]
UN_5

Sort order Major area, region, country or area of destination Major area  \
0             1                                              WORLD        NaN   
1             2                                  Developed regions        NaN   
2             3                                 Developing regions        NaN   
3             4                          Least developed countries        NaN   
4             5  Less developed regions excluding least develop...        NaN   
...         ...                                                ...        ...   
3970        261                                              Samoa    Oceania   
3971        262                                            Tokelau    Oceania   
3972        263                                              Tonga    Oceania   
3973        264                                             Tuvalu    Oceania   
3974        265                          Wallis and Futuna Islands    Oceania   

         Region Notes Country code Type of data (a) gender       year  \
0           NaN   NaN          900              NaN   Both  1990-1995   
1           NaN   (b)          901              NaN   Both  1990-1995   
2           NaN   (c)          902              NaN   Both  1990-1995   
3           NaN   (d)          941              NaN   Both  1990-1995   
4           NaN   NaN          934              NaN   Both  1990-1995   
...         ...   ...          ...              ...    ...        ...   
3970  Polynesia   NaN          882                B   Feml  2010-2015   
3971  Polynesia   NaN          772                B   Feml  2010-2015   
3972  Polynesia   NaN          776                B   Feml  2010-2015   
3973  Polynesia   NaN          798                C   Feml  2010-2015   
3974  Polynesia   NaN          876                B   Feml  2010-2015   

     Annual rate of change of the migrant stock Developed region  \
0                                      1.051865              NaN   
1                                      2.275847              NaN   
2                                     -0.487389              NaN   
3                                      1.118175              NaN   
4                                     -0.803244              NaN   
...                                         ...              ...   
3970                                  -0.545343               No   
3971                                    2.60325               No   
3972                                   2.526318               No   
3973                                  -1.819436               No   
3974                                   0.516899               No   

     Least developed country Sub-Saharan Africa  
0                        NaN                NaN  
1                        NaN                NaN  
2                        NaN                NaN  
3                        NaN                NaN  
4                        NaN                NaN  
...                      ...                ...  
3970                      No                 No  
3971                      No                 No  
3972                      No                 No  
3973                     Yes                 No  
3974                      No                 No  

[3930 rows x 13 columns]

In [11]:
#Similiar to UN_4
#Spit the table into 3 tables as they are 3 different obervations.
UN_6 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', sheet_name="Table 6")
UN_6 = UN_6.iloc[15:]
UN_6.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                '1-1990','1-1995','1-2000','1-2005','1-2010','1-2015',
                '2-1990','2-1995','2-2000','2-2005','2-2010','2-2015',
                '1990-1995','1995-2000','2000-2005','2005-2010','2010-2015']
UN_6_1 = UN_6[['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                '1-1990','1-1995','1-2000','1-2005','1-2010','1-2015']]
UN_6_1 = UN_6_1.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code','Type of data (a)'], 
                 var_name=["tmp"],value_name="Estimated refugee stock at mid-year (both sexes)")
UN_6_1 = UN_6_1.assign(year = lambda x: x.tmp.str[2:].astype(str)).drop("tmp",axis=1)
UN_6_1 = UN_6_1.merge(ANNEX, how='left', on=['Country code'])
UN_6_1 = UN_6_1.drop(columns=['Country or area', 'Sort order of Country', 
        'Code of Major area', 'Sort order of Major area',
        'Code of Region', 'Sort order of Region'])
UN_6_1

Sort order Major area, region, country or area of destination Notes  \
0             1                                              WORLD   NaN   
1             2                                  Developed regions   (b)   
2             3                                 Developing regions   (c)   
3             4                          Least developed countries   (d)   
4             5  Less developed regions excluding least develop...   NaN   
...         ...                                                ...   ...   
1585        261                                              Samoa   NaN   
1586        262                                            Tokelau   NaN   
1587        263                                              Tonga   NaN   
1588        264                                             Tuvalu   NaN   
1589        265                          Wallis and Futuna Islands   NaN   

     Country code Type of data (a)  \
0             900              NaN   
1             901              NaN   
2             902              NaN   
3             941              NaN   
4             934              NaN   
...           ...              ...   
1585          882                B   
1586          772                B   
1587          776                B   
1588          798                C   
1589          876                B   

     Estimated refugee stock at mid-year (both sexes)  year Major area  \
0                                            18836571  1990        NaN   
1                                             2014564  1990        NaN   
2                                            16822007  1990        NaN   
3                                             5048391  1990        NaN   
4                                            11773616  1990        NaN   
...                                               ...   ...        ...   
1585                                              0.0  2015    Oceania   
1586                                              0.0  2015    Oceania   
1587                                              0.0  2015    Oceania   
1588                                              0.0  2015    Oceania   
1589                                              0.0  2015    Oceania   

         Region Developed region Least developed country Sub-Saharan Africa  
0           NaN              NaN                     NaN                NaN  
1           NaN              NaN                     NaN                NaN  
2           NaN              NaN                     NaN                NaN  
3           NaN              NaN                     NaN                NaN  
4           NaN              NaN                     NaN                NaN  
...         ...              ...                     ...                ...  
1585  Polynesia               No                      No                 No  
1586  Polynesia               No                      No                 No  
1587  Polynesia               No                      No                 No  
1588  Polynesia               No                     Yes                 No  
1589  Polynesia               No                      No                 No  

[1590 rows x 12 columns]

In [10]:
UN_6_2 = UN_6[['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                '2-1990','2-1995','2-2000','2-2005','2-2010','2-2015']]
UN_6_2 = UN_6_2.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code','Type of data (a)'], 
                 var_name=["tmp"],value_name="Refugees as a percentage of the international migrant stock")
UN_6_2 = UN_6_2.assign(year = lambda x: x.tmp.str[2:].astype(str)).drop("tmp",axis=1)
UN_6_2 = UN_6_2.merge(ANNEX, how='left', on=['Country code'])
UN_6_2 = UN_6_2.drop(columns=['Country or area', 'Sort order of Country', 
        'Code of Major area', 'Sort order of Major area',
        'Code of Region', 'Sort order of Region'])
UN_6_2


Sort order Major area, region, country or area of destination Notes  \
0             1                                              WORLD   NaN   
1             2                                  Developed regions   (b)   
2             3                                 Developing regions   (c)   
3             4                          Least developed countries   (d)   
4             5  Less developed regions excluding least develop...   NaN   
...         ...                                                ...   ...   
1585        261                                              Samoa   NaN   
1586        262                                            Tokelau   NaN   
1587        263                                              Tonga   NaN   
1588        264                                             Tuvalu   NaN   
1589        265                          Wallis and Futuna Islands   NaN   

     Country code Type of data (a)  \
0             900              NaN   
1             901              NaN   
2             902              NaN   
3             941              NaN   
4             934              NaN   
...           ...              ...   
1585          882                B   
1586          772                B   
1587          776                B   
1588          798                C   
1589          876                B   

     Refugees as a percentage of the international migrant stock  year  \
0                                             12.346732           1990   
1                                              2.445494           1990   
2                                             23.968236           1990   
3                                              45.56588           1990   
4                                             19.919743           1990   
...                                                 ...            ...   
1585                                                0.0           2015   
1586                                                0.0           2015   
1587                                                0.0           2015   
1588                                                0.0           2015   
1589                                                0.0           2015   

     Major area     Region Developed region Least developed country  \
0           NaN        NaN              NaN                     NaN   
1           NaN        NaN              NaN                     NaN   
2           NaN        NaN              NaN                     NaN   
3           NaN        NaN              NaN                     NaN   
4           NaN        NaN              NaN                     NaN   
...         ...        ...              ...                     ...   
1585    Oceania  Polynesia               No                      No   
1586    Oceania  Polynesia               No                      No   
1587    Oceania  Polynesia               No                      No   
1588    Oceania  Polynesia               No                     Yes   
1589    Oceania  Polynesia               No                      No   

     Sub-Saharan Africa  
0                   NaN  
1                   NaN  
2                   NaN  
3                   NaN  
4                   NaN  
...                 ...  
1585                 No  
1586                 No  
1587                 No  
1588                 No  
1589                 No  

[1590 rows x 12 columns]

In [12]:
UN_6_3 = UN_6[['Sort order', 'Major area, region, country or area of destination', 'Notes', 
                'Country code','Type of data (a)',
                '1990-1995','1995-2000','2000-2005','2005-2010','2010-2015']]
UN_6_3 = UN_6_3.melt(id_vars=["Sort order","Major area, region, country or area of destination", "Notes",
                         'Country code','Type of data (a)'], 
                 var_name=["tmp"],value_name="Annual rate of change of the refugee stock")
UN_6_3 = UN_6_3.assign(year = lambda x: x.tmp.str[0:].astype(str)).drop("tmp",axis=1)
UN_6_3 = UN_6_3.merge(ANNEX, how='left', on=['Country code'])
UN_6_3 = UN_6_3.drop(columns=['Country or area', 'Sort order of Country', 
        'Code of Major area', 'Sort order of Major area',
        'Code of Region', 'Sort order of Region'])
UN_6_3 = UN_6_3[UN_6_3["Annual rate of change of the refugee stock"] != ".."]

# UN_6 = UN_6.merge(ANNEX, how='left', on=['Country code'])
# UN_6drop = UN_6.drop(columns=['Country or area', 'Sort order of Country', 
#        'Code of Major area', 'Sort order of Major area',
#        'Code of Region', 'Sort order of Region', "year1", "year2", "year0"])
# UN_6 = UN_6[['Sort order', 'Major area, region, country or area of destination',
#              'Major area', 'Region','Notes', 'Country code', 'Type of data (a)', 'year',
#              'Refugees as a percentage of the international migrant stock', "Estimated refugee stock at mid-year (both sexes)",
#              "gap_year", "Annual rate of change of the refugee stock",
#              'Developed region', 'Least developed country','Sub-Saharan Africa']]
UN_6_3

Sort order Major area, region, country or area of destination Notes  \
0             1                                              WORLD   NaN   
1             2                                  Developed regions   (b)   
2             3                                 Developing regions   (c)   
3             4                          Least developed countries   (d)   
4             5  Less developed regions excluding least develop...   NaN   
...         ...                                                ...   ...   
1300        241                                        New Zealand   NaN   
1301        242                                          Melanesia   NaN   
1302        243                                               Fiji   NaN   
1304        245                                   Papua New Guinea   NaN   
1306        247                                            Vanuatu   NaN   

     Country code Type of data (a) Annual rate of change of the refugee stock  \
0             900              NaN                                  -2.123497   
1             901              NaN                                   9.388424   
2             902              NaN                                  -2.839417   
3             941              NaN                                  -0.680327   
4             934              NaN                                    -4.3836   
...           ...              ...                                        ...   
1300          554                B                                  -11.80581   
1301          928              NaN                                  -0.268521   
1302          242                B                                  50.708582   
1304          598              C R                                   0.137417   
1306          548                B                                 -15.132389   

           year Major area                     Region Developed region  \
0     1990-1995        NaN                        NaN              NaN   
1     1990-1995        NaN                        NaN              NaN   
2     1990-1995        NaN                        NaN              NaN   
3     1990-1995        NaN                        NaN              NaN   
4     1990-1995        NaN                        NaN              NaN   
...         ...        ...                        ...              ...   
1300  2010-2015    Oceania  Australia and New Zealand              Yes   
1301  2010-2015        NaN                        NaN              NaN   
1302  2010-2015    Oceania                  Melanesia               No   
1304  2010-2015    Oceania                  Melanesia               No   
1306  2010-2015    Oceania                  Melanesia               No   

     Least developed country Sub-Saharan Africa  
0                        NaN                NaN  
1                        NaN                NaN  
2                        NaN                NaN  
3                        NaN                NaN  
4                        NaN                NaN  
...                      ...                ...  
1300                      No                 No  
1301                     NaN                NaN  
1302                      No                 No  
1304                      No                 No  
1306                     Yes                 No  

[890 rows x 12 columns]

In [ ]:
$